# Text Classification with PyTorch

In [34]:
import torch
import pandas as pd

In [35]:
df = pd.read_csv('../data/merged_plots.csv', index_col=0)
df

,title,plot,is_indian
0,10.0 Earthquake,As a series of minor earthquakes start tearing...,0
1,12 Rounds (film),A sting operation to capture arms dealer Miles...,0
2,12 Rounds 3: Lockdown,Detective Tyler Burke and his two men infiltr...,0
3,200 mph,When the older brother (Tommy Nash) he idolize...,0
4,Ablaze (2001 film),Andrew Thomas is an agent tasked with recordin...,0
...,...,...,...
3995,Choked (film),Sarita Pillai and Sushant Pillai are a married...,1
3996,Chumbak,"Chumbak is a coming-of-age story of Baalu, a t...",1
3997,Chungakkarum Veshyakalum,Chungakkarum Veshyakalum is the story of a Mal...,1
3998,Chuzhi,Varghese is a planter who lives with his wife ...,1


In [36]:
value_tuples = []
for row in df.iterrows():
    values = row[1]
    value_tuples.append((values['is_indian'], values['plot']))

print(len(value_tuples))
print(value_tuples[0])
print(value_tuples[-1])

4000
(0, 'As a series of minor earthquakes start tearing apart Los Angeles, scientist Emily of the USGS theorizes that it is all building to a super quake that will drop the entire city into a lava-filled chasm. Engineer Jack whose daughter has gone camping with her friends in the danger area and whose company is responsible for the quakes due to their deep fracking feels obligated to help, and races with Emily through the increasingly damaged city with the hopes of diverting the epicenter to Long Beach and potentially saving millions of lives in the city of Los Angeles.')
(1, 'Jaya is a cinephile. So obsessed is she with cinema, that she worships the popular actor Jaishankar, believing all that he portrays on screen, to be his real self. He becomes the ideal man of her dreams, and she even tattoos his name in her arm. Hell breaks loose when she refuses to marry Natarajan, the boy arranged by her brother and her sister-in-law. At his juncture, steps in her uncle, who takes it upon hims

In [37]:
from sklearn.model_selection import train_test_split

train_value_tuples, test_value_tuples = train_test_split(value_tuples, test_size=0.3, shuffle=True, random_state=0)

print('training size:', len(train_value_tuples))
print('testing size:', len(test_value_tuples))

training size: 2800
testing size: 1200


## Preparing data processing pipelines

In [38]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

train_iter = iter(train_value_tuples)

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [39]:
# # Named entities, stop words and punctuation are STILL part of the vocabulary

example_text = 'So obsessed is she with cinema, that she worships the popular actor Jaishankar'
print('raw:',example_text)
print('cleaned:',tokenizer(example_text))
print('bow:',vocab(tokenizer(example_text)))
print('\nvocab length:', len(vocab))

raw: So obsessed is she with cinema, that she worships the popular actor Jaishankar
cleaned: ['so', 'obsessed', 'is', 'she', 'with', 'cinema', ',', 'that', 'she', 'worships', 'the', 'popular', 'actor', 'jaishankar']
bow: [87, 4032, 10, 27, 14, 5172, 2, 15, 27, 12000, 1, 2007, 1886, 33212]

vocab length: 41544


In [40]:
text_pipeline = lambda x: vocab(tokenizer(x))

# label pipeline does not have to do anything since values are already 0 or 1
label_pipeline = lambda x: int(x)

In [41]:
text_pipeline('here is an natarajan')

[934, 10, 29, 35459]

In [42]:
label_pipeline(0)

0

## Generate data batch and iterator

In [43]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

## Define the model

In [44]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## Initiate an Instance

In [45]:
train_iter = iter(train_value_tuples)
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

## Functions for evaluation and training

In [46]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Running the model

In [47]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter = iter(train_value_tuples)
test_iter = iter(test_value_tuples)

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  4.16s | valid accuracy    0.607 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  3.38s | valid accuracy    0.764 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  3.04s | valid accuracy    0.686 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  3.15s | valid accuracy    0.814 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  3.23s | valid accuracy    0.821 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  3.87s |

## Evaluating the model

In [48]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.828
